MCDA 코드 분석

In [1]:
#env 로드
import os
import topogenesis as tg
import trimesh as tm
import numpy as np
import pyvista as pv
import panel as pn
import pandas as pd 
import copy

from ladybug.sunpath import Sunpath #ladybug sunpath 호출
from IPython.display import display, Image

In [2]:
# csv에서 lattice 배열 만드는 함수 정의
def lattice_from_csv(file_path):
    # 데이터를 읽어옴
    meta_df = pd.read_csv(file_path, nrows=3)
    # shape, unit, minbound 데이터를 배열로 변환
    shape = np.array(meta_df['shape'])       # 시작 좌표 [x_min, y_min, z_min] 예를 들어, [-55000, -7000, 0] x축 -55000, y축 -7000, z축 0의 위치
    unit = np.array(meta_df['unit'])         # 몇개의 셀로 구성되는지 [x_shape, y_shape, z_shape] 예를 들어 [56, 35, 24]이면 x축 56개, y 35개, z 24개의 셀을 가집니다.
    minbound = np.array(meta_df['minbound']) # 각 셀의 크기. 예를 들어, [1000, 1000, 3000] 각 셀의 크기는 x축 1000 단위, y축 1000 단위, z축 3000 단위
    # np.array는 csv에서 불러온 데이터를 배열로 변환하는데 사용 
    # 그리드 데이터를 읽어옴
    lattice_df = pd.read_csv(file_path, skiprows=5) # 앞의 5개 줄을 스킵하고 6개 부터 데이터를 불러오기
    # 데이터의 'value' 열을 배열로 변환한 후 이를 'shape' 배열에 맞게 재구성 
    buffer = np.array(lattice_df['value']).reshape(shape)
    # lattice를 토포제네시스에서 재구성 buffer는 배열에 맞는 value값. 
    l = tg.to_lattice(buffer, minbound=minbound, unit=unit)
    
    return l

In [3]:
# tri_to_pv 함수 정의: 삼각형 메시(tri_mesh)를 PyVista 형식으로 변환합니다.
def tri_to_pv(tri_mesh):
    #  tri_mesh.faces 배열에 각 face의 시작 부분에 3을 추가하여 새로운 배열을 만듭니다.
    # 'constant' 모드로 패딩을 추가하여 각 face의 시작에 3을 삽입합니다.
    faces = np.pad(tri_mesh.faces, ((0, 0), (1, 0)), 'constant', constant_values=3)
    # PyVista의 PolyData 객체를 생성하여, tri_mesh의 정점(vertices)과 패딩된 faces 배열을 사용합니다.
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)    
    # 변환된 PyVista 메쉬 객체를 반환합니다.
    return pv_mesh

In [4]:
#csv 로드
csv_path = os.path.relpath('original_lattice/interior_lattice.csv')
#obj 포인트 csv 로드
context_mesh = tm.load(os.path.relpath('obj\context.obj'))  # 주변환경 obj 메시 불러오기
street_pc = tg.cloud_from_csv("pts\pts_str(11).CSV")        # street point as csv (포인트를 csv로 저장) 
entrance_pc = tg.cloud_from_csv("pts\ent_pts01.CSV")        # entrance point as csv (라이노 포인트를 csv로 저장)
envelope_lattice = tg.lattice_from_csv(csv_path)

In [5]:
avail_lattice = tg.lattice_from_csv(csv_path)                                         # CSV 파일에서 격자 데이터 불러오기
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)             # 복사된 데이터를 격자 객체로 변환

### 이웃 패턴(Neighborhood Pattern) 정의

구조에서 특정 셀과 `이웃 관계`를 가지는 셀들의 집합을 정의하는 방식

- **격자(Lattice)**: 3차원 공간을 작은 셀로 나눈 구조입니다.
- **셀(Cell)**: 격자를 구성하는 가장 작은 단위입니다.
- **이웃(Neighbor)**: 특정 셀과 인접한 셀들입니다.

#### 패턴의 종류

`von Neumann 패턴`
- 셀의 상하좌우에 위치한 셀들을 이웃으로 간주합니다.
```markdown
  [ ] [ ] [ ]    [ ] [1] [ ]
  [ ] [R] [ ] => [2] [R] [3]
  [ ] [ ] [ ]    [ ] [4] [ ]
```

`Moore 패턴`
- 셀의 대각선 방향까지 포함하여 더 많은 셀들을 이웃으로 간주합니다
```markdown
  [ ] [ ] [ ]    [1] [2] [3]
  [ ] [R] [ ] => [4] [R] [5]
  [ ] [ ] [ ]    [6] [7] [8]
```



In [6]:
"""패턴1"""
# 1이라는 범위에 대한 이웃패턴 정의 (중심을 포함함)
stencil = tg.create_stencil("von_neumann", 1, 1)
# 중심을 인덱스 비활성화 - [0,0,0]을 0으로 세팅함으로 중심을 패턴 비활성화
stencil.set_index([0,0,0], 0)

""" 패턴2 - squareness 가로로 agent가 땅따먹을 수 있도록 이웃패턴2 정의 """
stencil_sq = tg.create_stencil("von_neumann", 1, 1)
# 중심 인덱스 비활성화  
stencil_sq.set_index([0,0,0], 0)
# y축 방향 비활성화
stencil_sq.set_index([0,1,0], 1)
stencil_sq.set_index([0,-1,0], 1)
# z축 방향 비활성화
stencil_sq.set_index([0,0,1], 0)
stencil_sq.set_index([0,0,-1], 0)
# x축 방향 활성화
stencil_sq.set_index([1,0,0], 1)
stencil_sq.set_index([-1,0,0], 1)

print(stencil_sq)

[[[0 0 0]
  [0 1 0]
  [0 0 0]]

 [[0 1 0]
  [0 0 0]
  [0 1 0]]

 [[0 0 0]
  [0 1 0]
  [0 0 0]]]


In [7]:
# # PyVista 플로터 초기화
# p = pv.Plotter(notebook=True, window_size=[600, 200])  # 가로 800, 세로 500 픽셀
# # PyVista ImageData 객체 생성 (pyvista에 imagedata객체 생성)
# grid = pv.ImageData()
# # 배열의 차원 설정 (stencil_sq의 형태에 1을 더한 값)
# grid.dimensions = np.array(stencil_sq.shape) + 1
# grid.origin = [0, 0, 0]   # 배열의 시작점 설정
# grid.spacing = [1, 1, 1]   # 배열의 셀 크기 설정 

# # `stencil_sq는 3차원의 배열`grid 객체에 이를 저장하려면 1차원의 배열로 변환해야함
# # 예를 들어 stencil_sq = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
# # 포트란 스타일(order="F")로 평탄화하면: [1, 5, 3, 7, 2, 6, 4, 8]
# grid.cell_data["values"] = stencil_sq.flatten(order="F")

# # 배열 데이터에서 값이 0.9와 1.1 사이에 있는 셀들 `1`을 선택
# threshed = grid.threshold([0.9, 1.1])
# # 값을 메쉬로 추가, 가장자리 표시, 색상 및 불투명도 설정
# p.add_mesh(threshed, show_edges=True, color="#ff8fa3", opacity=0.3)
# p.show(jupyter_backend="html", return_viewer=True) # 플로터 표시

In [8]:
# 'excel/program (이노션).xlsx' 파일에서 'Sheet1' 시트를 읽어옴
df = pd.read_excel('excel/program (이노션).xlsx', sheet_name='Sheet1')

# 선택한 열을 복사하여 program_prefs 데이터프레임 생성
# [sun_acc 태양 근접 선호도], [ent_acc 입구 근접 선호도], [str_acc 도로 근접 선호도], [ung_pre 지하 선호도], [top_pre 탑층 선호도] 
program_prefs = df[['sun_acc', 'ent_acc', 'str_acc', 'ung_pre', 'dist_façade','top_pre']].copy()

# 인덱스를 재설정(기존 index를 버리고 새로운 인덱스로) 0부터 숫자 인덱스를 생성
program_prefs.reset_index(drop=True, inplace=True)
# program_prefs 데이터프레임 출력 (주석 처리됨)
print(program_prefs)

    sun_acc  ent_acc  str_acc  ung_pre  dist_façade  top_pre
0       0.2      0.9      0.9      0.1          0.1      0.1
1       0.3      0.9      0.9      0.2          0.2      0.1
2       0.2      0.9      0.9      0.2          0.2      0.1
3       0.2      0.8      0.8      0.2          0.2      0.4
4       0.5      0.5      0.5      0.1          0.3      0.4
5       0.3      0.2      0.3      0.1          0.5      0.4
6       0.1      0.2      0.5      0.1          0.9      0.4
7       0.5      0.2      0.2      0.1          0.5      0.8
8       0.5      0.5      0.2      0.1          0.6      0.6
9       0.2      0.1      0.2      0.1          0.8      0.6
10      0.2      0.1      0.2      0.1          0.9      0.6
11      0.6      0.1      0.2      0.1          0.2      0.6
12      0.5      0.1      0.2      0.1          0.2      0.6
13      0.3      0.1      0.2      0.1          0.2      0.6
14      0.3      0.1      0.2      0.1          0.2      0.6
15      0.3      0.1    

In [9]:
fields = {}  # 필드를 저장할 빈 딕셔너리를 생성

# program_pref 각 필드에 대해 반복 (5개 열)
for f in program_prefs.columns:  
    # 각 필드에 대한 파일 경로 설정
    lattice_path = os.path.relpath('matrixs/' + f + '.csv')      
    try:
        # 배열 파일에서 데이터를 불러와서 fields 딕셔너리에 저장
        fields[f] = tg.lattice_from_csv(lattice_path)  
    except:
        # 파일이 없을 경우, 모든 값을 1로 설정한 기본 격자 데이터를 fields 딕셔너리에 저장
        fields[f] = copy.deepcopy(avail_lattice * 0 + 1)

# 'top_pre' 열의 인덱스 찾기 (5번째에 위치)
top_pre_index = df.columns.get_loc('top_pre')
# 'top_pre' 열 이후 모든 열을 포함하는 새로운 데이터프레임 생성 (프로그램 상관관계 matrix 찾기)
program_mtx = df.iloc[:, top_pre_index + 1:]
# 새로운 데이터프레임의 열 이름을 0부터 시작하는 숫자로 변경     (프로그램 번호입력 0~ ) 
program_mtx.columns = range(len(program_mtx.columns))

# program_prefs의 복사본 생성 (원본 데이터를 건드리지 않게)
program_prefs_copy = program_prefs.copy()
# program_prefs_copy에 'space_id' 열 추가 (원본 df의 인덱스 사용)
program_prefs_copy['space_id'] = df.index
# # # 인덱스를 재설정하여 연속적인 숫자 인덱스를 생성
program_prefs_copy.reset_index(drop=True, inplace=True)

# 최종 데이터프레임 생성
program_complete = program_prefs_copy



In [10]:
# 'space_name' 열을 딕셔너리로 변환하여 'space_list'에 저장
space_list = df['space_name'].to_dict()
# 'vox_amount' 열의 값을 가져와 'sizes_complete'에 저장
sizes_complete = df['vox_amount']
# 'agent_areas' 리스트 초기화
agent_areas = []
# 'sizes_complete'의 각 값에 대해 반복
for area in sizes_complete:
    # 각 값을 정수로 변환하고 반올림하여 'agent_areas'에 추가
    agent_areas.append(round(int(area)))

print(space_list)   # 변환된 딕셔너리를 출력
print(agent_areas)  # 'agent_areas' 리스트를 출력

{0: 'Lobby', 1: 'Branding Space', 2: 'External Contact', 3: 'Interview Room', 4: 'Personal Workspace', 5: 'Work Support Area', 6: 'Personal Locker Room', 7: 'Executive Space', 8: 'VIP Reception', 9: 'Secretary Room', 10: 'Pantry', 11: 'Executive Meeting', 12: 'Executive Offices', 13: 'Focus Rooms', 14: 'Project Rooms', 15: 'Conference Rooms', 16: '4-Person Meeting', 17: '6-Person Meeting', 18: '8-Person Meeting', 19: '12-Person Meeting', 20: '18-Person Meeting', 21: 'Idea Room', 22: 'Mail Room', 23: 'Oasis', 24: 'PT Practice Room', 25: 'Screening Room', 26: 'Studio', 27: 'Video Editing', 28: 'Plotter/Board Room', 29: 'Training Room', 30: 'Ino Kitchen', 31: 'Kitchen', 32: 'Ino Cafe', 33: 'Fitness Facilities', 34: 'Dressing/Shower', 35: 'Counseling Room', 36: 'Personal Rest Room', 37: 'Sleeping Room', 38: 'Nursing Room', 39: 'Work Lounge', 40: 'Town Hall', 41: 'Seminar Room', 42: 'Library', 43: 'Social Space', 44: 'Outdoor Terrace', 45: "Driver's Waiting", 46: 'Server Room', 47: 'Cleanin

In [11]:
def initialize_agents(avail_lattice, program_prefs, fields, max_agents_per_floor=4):   
    """에이전트에 시작 위치 찾기 1단계 """
    
    agn_num = len(program_complete) 
    # 점유 배열(occupation lattice)를 초기화 (모든 값을 -1로 설정)
    occ_lattice = avail_lattice * 0 - 1
    # 층별 에이전트 수를 추적하는 배열 초기화
    floor_counts = np.zeros(avail_lattice.shape[2], dtype=int)
    # 에이전트 위치를 저장할 빈 리스트 생성
    agn_locs = [None]*agn_num

    # 각 에이전트의 시작 위치를 설정 (program_prefs 행 반복)
    for a_id, a_prefs in program_prefs.iterrows(): # a_id는 index (0,1,2,...) a_prefs는 행의 데이터 
        # 선호 배열(preference lattice) 초기화 (모든 값을 0으로 설정)
        pref_lattice = avail_lattice * 0.0

        # 각 필드에 대해, 가중치를 적용하여 선호 격자를 업데이트
        for f, w in a_prefs.items():
            pref_lattice += fields[f] * w

        # 선호 배열에서 최대값을 가지는 인덱스 선택
        select_id = np.argmax(pref_lattice)
        # 평평한(1차원) 배열의 특정 위치를 다시 원래의 다차원 배열의 위치로 변환
        a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
        
        # 초기 위치를 계산된 위치로 설정 (z 축 값을 그대로 유지)
        a_origin_2 = a_origin_1
        
        # 동일한 층에 에이전트가 너무 많이 배치되지 않도록 조정
        while avail_lattice[a_origin_2] == 0 or floor_counts[a_origin_2[2]] >= max_agents_per_floor:
            pref_lattice[a_origin_2] = -1  # 현재 위치를 -1로 설정하여 다음 최고 값을 선택하도록
            select_id = np.argmax(pref_lattice)
            a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
            a_origin_2 = a_origin_1

        # 에이전트 위치 리스트에 추가
        agn_locs[a_id] = a_origin_2
        # 가용 배열에서 해당 위치를 0으로 설정 (이제 사용 불가능)
        avail_lattice[a_origin_2] = 0
        # 점유 배열에서 해당 위치를 에이전트 ID로 설정
        occ_lattice[a_origin_2] = a_id
        # 해당 층의 에이전트 수를 증가
        floor_counts[a_origin_2[2]] += 1

    return occ_lattice, agn_locs

# 각 층에 최대 4개의 에이전트만 배치되도록 설정
occ_lattice, agn_locs = initialize_agents(avail_lattice, program_prefs, fields, max_agents_per_floor=4)

print("Occupation lattice:")
print(occ_lattice)
print("Agent locations:")
print(agn_locs)


Occupation lattice:
[[[-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 ...

 [[-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1

In [12]:

# def initialize_agents(avail_lattice, program_prefs, fields):   
#     """에이전트에 시작 위치 찾기 1단계 """
    
#     agn_num = len(program_complete) 
#     # 점유 배열(occupation lattice)를 초기화 (모든 값을 -1로 설정) 모든값을 0으로 곱한뒤 - 1 
#     occ_lattice = avail_lattice * 0 - 1
#     # 에이전트 위치를 저장할 빈 리스트 생성
#     agn_locs = [None]*agn_num
#     # 각 에이전트의 시작 위치를 설정 (program_prefs 행 반복)
#     for a_id, a_prefs in program_prefs.iterrows(): # a_id는 index (0,1,2,...) a_pref는 행의 데이터 
#         # 선호 배열(preference lattice) 초기화 모든값을 1로 설정 -> (가용 격자 값을 기반으로 설정)
#         pref_lattice = (avail_lattice * 0.0 + 1.0) * avail_lattice

#         # 각 필드에 대해, 가중치를 적용하여 선호 격자를 업데이트
#         for f, w in a_prefs.items():
#             pref_lattice *= fields[f] ** w

        
#         # 선호 배열에서 최대값을 가지는 인덱스 선택
#         select_id_wgt = np.argmax(pref_lattice)
#         # 평평한(1차원) 배열의 특정 위치를 다시 원래의 다차원 배열의 위치로 변환
#         a_origin_1 = np.unravel_index(select_id_wgt, avail_lattice.shape)
#         #a_origin_2_wgt = (a_origin_1_wgt[0], a_origin_1_wgt[1], 1)

#         # 선호 배열에서 최대값을 가지는 인덱스 선택
#         select_id = np.argmax(pref_lattice)
#         # 평평한(1차원) 배열의 특정 위치를 다시 원래의 다차원 배열의 위치로 변환
#         a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
        
       
#         # 초기 위치를 계산된 위치로 설정 (z 축 값을 그대로 유지)
#         #a_origin_2 = a_origin_1  # 또는 a_origin_1을 그대로 사용
#         a_origin_2 = (a_origin_1[0], a_origin_1[1], 1)

#         # 동일한 위치에 여러 에이전트가 배치되지 않도록 조정
#         for n in range(5):
#             if avail_lattice[a_origin_2] == 0:
#                 a_origin_2 = (a_origin_1[0], a_origin_1[1], a_origin_1[2])

#         # 에이전트 위치 리스트에 추가
#         agn_locs.append([a_origin_2])
#         # 가용 배열에서 해당 위치를 0으로 설정 (이제 사용 불가능)
#         avail_lattice[a_origin_2] = 0
#         # 점유 배열에서 해당 위치를 에이전트 ID로 설정
#         occ_lattice[a_origin_2] = a_id

#     return occ_lattice, agn_locs

# occ_lattice, agn_locs = initialize_agents(avail_lattice, program_complete, fields)

In [13]:
# def initialize_agents(avail_lattice, program_prefs, fields):   
#     """에이전트에 시작 위치 찾기 1단계 """
    
#     agn_num = len(program_complete) 
#     # 점유 배열(occupation lattice)를 초기화 (모든 값을 -1로 설정) 모든값을 0으로 곱한뒤 - 1 
#     occ_lattice = avail_lattice * 0 - 1
#     # 에이전트 위치를 저장할 빈 리스트 생성
#     agn_locs = [None]*agn_num
#     # 각 에이전트의 시작 위치를 설정 (program_prefs 행 반복)
#     for a_id, a_prefs in program_prefs.iterrows(): # a_id는 index (0,1,2,...) a_pref는 행의 데이터 
#         # 선호 배열(preference lattice) 초기화 모든값을 1로 설정 -> (가용 격자 값을 기반으로 설정)
#         pref_lattice = (avail_lattice * 0 + 1.0) * avail_lattice

#         # 각 필드에 대해, 가중치를 적용하여 선호 격자를 업데이트
#         for f, w in a_prefs.items(): #f 는 항목의 이름(열의 이름:sun_acc,ent_acc) w는 선호도의 값 (0.5, ... )
#             # '*='는 곱하기 할당, 변수에 곱한 결과를 다시 변수에 할당
#             # 'w' 가중치로 에이전트의 선호도를 반영 [0.6**0.2] 이런식
#             pref_lattice *= fields[f] ** w

#         # 선호 배열에서 최대값을 가지는 인덱스 선택
#         select_id = np.argmax(pref_lattice)
#         # 평평한(1차원) 배열의 특정 위치를 다시 원래의 다차원 배열의 위치로 변환
#         a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
        
       
#         # 초기 위치를 계산된 위치로 설정 (z 축 값을 그대로 유지)
#         a_origin_2 = a_origin_1  # 또는 a_origin_1을 그대로 사용
#         a_origin_2 = (a_origin_1[0], a_origin_1[1], 1)

#         # 동일한 위치에 여러 에이전트가 배치되지 않도록 조정
#         for n in range(5):
#             if avail_lattice[a_origin_2] == 0:
#                 a_origin_2 = (a_origin_1[0], a_origin_1[1], a_origin_1[2])

#         # 에이전트 위치 리스트에 추가
#         agn_locs.append([a_origin_2])
#         # 가용 배열에서 해당 위치를 0으로 설정 (이제 사용 불가능)
#         avail_lattice[a_origin_2] = 0
#         # 점유 배열에서 해당 위치를 에이전트 ID로 설정
#         occ_lattice[a_origin_2] = a_id

#     return occ_lattice, agn_locs

# occ_lattice, agn_locs = initialize_agents(avail_lattice, program_complete, fields)

In [14]:
from matplotlib.colors import ListedColormap
import pyvista as pv

# Visualization setup
p = pv.Plotter(notebook=True)

# Set the grid dimensions
grid = pv.ImageData()
grid.dimensions = np.array(occ_lattice.shape) + 1
# grid.origin과 grid.spacing 설정에 avail_lattice의 속성 사용
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# Adding the bounding box wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")
p.add_axes()
p.show_bounds(grid="back", location="back", color="#777777")

# Add the data values to the cell data
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!

# Filtering the voxels
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

colors = [
    "#FFB6C1", "#FFC0CB", "#FF69B4", "#FF1493",  # 분홍색 계열
    "#87CEEB", "#00BFFF", "#1E90FF",  # 파란색 계열
    "#98FB98", "#00FF7F", "#32CD32", "#3CB371", "#2E8B57", "#228B22",  # 녹색 계열
    "#FFD700", "#FFA500", "#FF8C00", "#FF7F50", "#FF6347", "#FF4500", "#FF0000", "#DC143C", "#B22222",  # 노란색 계열
    "#BA55D3", "#9932CC", "#9400D3", "#8A2BE2", "#9370DB", "#7B68EE", "#6A5ACD",  # 자주색 계열
    "#FFE4B5", "#FFDAB9", "#FFDEAD",  # 베이지 계열
    "#F0E68C", "#EEE8AA", "#F5DEB3", "#DEB887", "#D2B48C", "#BC8F8F",  # 갈색 계열
    "#B0C4DE", "#ADD8E6", "#B0E0E6", "#AFEEEE", "#00CED1",  # 연한 파란색 계열
    "#696969",  # 회색 계열
    "#708090",  # 슬레이트 그레이 계열
    "#4682B4", "#5F9EA0", "#6495ED", "#7B68EE", "#87CEFA", "#87CEFA"  # 추가 파란색 계열
]

custom_cmap = ListedColormap(colors)
# Scalar bar 설정
sargs = dict(
    shadow=True,
    n_labels=agn_num,
    italic=False,
    fmt=" %.0f",
    font_family="arial",
    height=0.6,
    vertical=True,

    title_font_size=10,  # 제목 폰트 크기 설정
    label_font_size=6   # 라벨 폰트 크기 설정
)
annotations = {i: name for i, name in enumerate(space_list.values())}
# Adding the voxels with the custom colormap and scalar bar arguments
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)
street_pc.fast_notebook_vis(p)
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)
#envelope_lattice.fast_vis(p)
# Show the plot
p.show(jupyter_backend="trame", return_viewer=True)  # 시각화 표시

Widget(value='<iframe src="http://localhost:62063/index.html?ui=P_0x1be5fcd7090_0&reconnect=auto" class="pyvis…

In [15]:
from matplotlib.colors import ListedColormap

# PyVista 플로터 초기화
p = pv.Plotter(notebook=True)

# 그리드 설정
#base_lattice = frames[0]
grid = pv.ImageData()
grid.dimensions = np.array(occ_lattice.shape) + 1
# grid.origin과 grid.spacing 설정에 avail_lattice의 속성 사용
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# 데이터 할당
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)

# 필터링
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

# 초기 컬러맵 설정
initial_colors = ["#808080" if i == 0 else "#FF0000" for i in range(len(program_prefs))]
custom_cmap = ListedColormap(initial_colors)

# 스칼라 바 설정
annotations = {i: name for i, name in enumerate(space_list.values())}
sargs = {
    "shadow": True,
    "n_labels": len(program_prefs),
    "italic": False,
    "fmt": " %.0f",
    "font_family": "arial",
    "height": 1,
    "vertical": True,
    "title_font_size": 10,
    "label_font_size": 6
}

# 메시 추가
opacity_map = np.full(threshed.n_cells, 0.1)  # 초기 투명도 설정
threshed.cell_data['opacity'] = opacity_map
mesh = p.add_mesh(threshed, scalars='opacity', cmap=custom_cmap, show_scalar_bar=True, scalar_bar_args=sargs, show_edges=True, opacity=1)

# 텍스트 추가
text_actor = p.add_text("Selected Program: None", position='upper_right', font_size=12, color='black')

# 슬라이더 콜백 함수
def slider_callback(value):
    selected_program = int(value)
    mask = threshed.cell_data['Agents'] == selected_program
    opacity = np.full(threshed.n_cells, 0.1)  # 기본 투명도 설정
    opacity[mask] = 1.0  # 선택된 프로그램에 높은 투명도 부여

    # 투명도 업데이트
    threshed.cell_data['opacity'] = opacity
    
    # 텍스트 업데이트
    text_actor.SetText(0, f"Selected Program: {annotations[selected_program]}")
    p.render()

# 슬라이더 위젯 추가
p.add_slider_widget(slider_callback, [0, len(program_prefs) - 1], value=0, title='Program Index')
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)

p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)
# 시각화 표시
p.show(jupyter_backend="trame", return_viewer=True)

Widget(value='<iframe src="http://localhost:62063/index.html?ui=P_0x1be5fce6dd0_1&reconnect=auto" class="pyvis…

In [16]:
def find_optimal_location_for_agent(agent_id, agent_prefs, avail_lattice, fields, stencil, occ_lattice, max_attempts=100):
    """에이전트에 대한 최적의 시작 위치 찾기 (2단계)"""
    
    # 가용 배열의 인덱스들을 배열로 변환 
    # np.where() 배열에서 'True'인 요소의 인덱스를 반환 = avail_lattice 배열에서 True 값을 가지는 위치 / 반환값은 세 개의 1차원 배열, 각각 x, y, z 축의 인덱스를 나타냅니다.
    # np.array() 반환된 인덱스 배열을 하나의 2차원 배열로 (행렬 형태)
    avail_index = np.array(np.where(avail_lattice)).T 
    # 에이전트 평가 배열을 1로 초기화 avail_index의 배열의 길이로, np.ones 그 길이만큼 모든요소가 1인 배열 생성
    a_eval = np.ones(len(avail_index))

    # 각 선호도 열에 대해 반복
    for f in program_prefs.columns:
        # 현재 선호도 열에 해당하는 필드 값들을 가져옴
        vals = fields[f][avail_index[:, 0], avail_index[:, 1], avail_index[:, 2]]
        # 값이 무한대인 경우 0으로 설정하고, 선호도 가중치를 적용
        a_weighted_vals = np.where(np.isinf(vals), 0, vals ** agent_prefs[f])
        # 평가 배열에 가중치 적용된 값을 곱함
        a_eval *= a_weighted_vals

    # 평가 배열을 기준으로 인덱스를 내림차순으로 정렬
    sorted_indices = np.argsort(-a_eval)
    # 평가 배열을 기준으로 인덱스를 내림차순으로 정렬
    sorted_indices = np.argsort(-a_eval)
    attempts = 0

    for selected_int in sorted_indices:
        if attempts >= max_attempts:
            break  # 최대 시도 횟수를 초과하면 종료

        selected_ind = avail_index[selected_int]

        while True:
            fns = avail_lattice.find_neighbours_masked(stencil, loc=selected_ind)
            blocked = 0
            for n in fns:
                neigh_3d_id = np.unravel_index(n, avail_lattice.shape)
                if occ_lattice[neigh_3d_id] != -1:
                    blocked += 1

            if blocked >= 2:
                selected_int += 1
                if selected_int >= len(sorted_indices):
                    break  # 선택할 수 있는 인덱스를 초과하면 종료
                selected_ind = avail_index[sorted_indices[selected_int]]
            else:
                print(f"Optimal location for agent {agent_id} found at {selected_ind}")
                return selected_ind

        attempts += 1

    # 적합한 위치를 찾지 못한 경우
    print(f"No suitable location found for agent {agent_id} after {attempts} attempts")    
    return None

In [17]:
# 각 에이전트의 최적 위치를 찾기
for a_id, a_prefs in program_prefs.iterrows():
    # 각 에이전트에 대해 최적의 위치를 찾음
    optimal_location = find_optimal_location_for_agent(a_id, a_prefs, avail_lattice, fields, stencil, occ_lattice)
    if optimal_location is not None:
        print(f"Agent {space_list[a_id]} optimal location: {optimal_location}")
    else:
        print(f"Agent {space_list[a_id]} has no suitable location.")

Optimal location for agent 0 found at [31  8  5]
Agent Lobby optimal location: [31  8  5]
Optimal location for agent 1 found at [46  8  5]
Agent Branding Space optimal location: [46  8  5]
Optimal location for agent 2 found at [31  8  5]
Agent External Contact optimal location: [31  8  5]
Optimal location for agent 3 found at [32 28 21]
Agent Interview Room optimal location: [32 28 21]
Optimal location for agent 4 found at [46  8 10]
Agent Personal Workspace optimal location: [46  8 10]
Optimal location for agent 5 found at [46  9 12]
Agent Work Support Area optimal location: [46  9 12]
Optimal location for agent 6 found at [46  8 10]
Agent Personal Locker Room optimal location: [46  8 10]
Optimal location for agent 7 found at [27 10 18]
Agent Executive Space optimal location: [27 10 18]
Optimal location for agent 8 found at [46  9 12]
Agent VIP Reception optimal location: [46  9 12]
Optimal location for agent 9 found at [17  8 17]
Agent Secretary Room optimal location: [17  8 17]
Opti

In [18]:
from matplotlib.colors import ListedColormap
import pyvista as pv

# Visualization setup
p = pv.Plotter(notebook=True)

# Set the grid dimensions
grid = pv.ImageData()
grid.dimensions = np.array(occ_lattice.shape) + 1
# grid.origin과 grid.spacing 설정에 avail_lattice의 속성 사용
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# Adding the bounding box wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")
p.add_axes()
p.show_bounds(grid="back", location="back", color="#777777")

# Add the data values to the cell data
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!

# Filtering the voxels
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

colors = [
    "#FFB6C1", "#FFC0CB", "#FF69B4", "#FF1493",  # 분홍색 계열
    "#87CEEB", "#00BFFF", "#1E90FF",  # 파란색 계열
    "#98FB98", "#00FF7F", "#32CD32", "#3CB371", "#2E8B57", "#228B22",  # 녹색 계열
    "#FFD700", "#FFA500", "#FF8C00", "#FF7F50", "#FF6347", "#FF4500", "#FF0000", "#DC143C", "#B22222",  # 노란색 계열
    "#BA55D3", "#9932CC", "#9400D3", "#8A2BE2", "#9370DB", "#7B68EE", "#6A5ACD",  # 자주색 계열
    "#FFE4B5", "#FFDAB9", "#FFDEAD",  # 베이지 계열
    "#F0E68C", "#EEE8AA", "#F5DEB3", "#DEB887", "#D2B48C", "#BC8F8F",  # 갈색 계열
    "#B0C4DE", "#ADD8E6", "#B0E0E6", "#AFEEEE", "#00CED1",  # 연한 파란색 계열
    "#696969",  # 회색 계열
    "#708090",  # 슬레이트 그레이 계열
    "#4682B4", "#5F9EA0", "#6495ED", "#7B68EE", "#87CEFA", "#87CEFA"  # 추가 파란색 계열
]

custom_cmap = ListedColormap(colors)
# Scalar bar 설정
sargs = dict(
    shadow=True,
    n_labels=agn_num,
    italic=False,
    fmt=" %.0f",
    font_family="arial",
    height=0.6,
    vertical=True,

    title_font_size=10,  # 제목 폰트 크기 설정
    label_font_size=6   # 라벨 폰트 크기 설정
)
annotations = {i: name for i, name in enumerate(space_list.values())}
# Adding the voxels with the custom colormap and scalar bar arguments
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)
street_pc.fast_notebook_vis(p)
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)
#envelope_lattice.fast_vis(p)
# Show the plot
p.show(jupyter_backend="trame", return_viewer=True)  # 시각화 표시



Widget(value='<iframe src="http://localhost:62063/index.html?ui=P_0x1be5fce6150_2&reconnect=auto" class="pyvis…

In [19]:
from matplotlib.colors import ListedColormap

# PyVista 플로터 초기화
p = pv.Plotter(notebook=True)

# 그리드 설정
#base_lattice = frames[0]
grid = pv.ImageData()
grid.dimensions = np.array(occ_lattice.shape) + 1
# grid.origin과 grid.spacing 설정에 avail_lattice의 속성 사용
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# 데이터 할당
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)

# 필터링
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

# 초기 컬러맵 설정
initial_colors = ["#808080" if i == 0 else "#FF0000" for i in range(len(program_prefs))]
custom_cmap = ListedColormap(initial_colors)

# 스칼라 바 설정
annotations = {i: name for i, name in enumerate(space_list.values())}
sargs = {
    "shadow": True,
    "n_labels": len(program_prefs),
    "italic": False,
    "fmt": " %.0f",
    "font_family": "arial",
    "height": 1,
    "vertical": True,
    "title_font_size": 10,
    "label_font_size": 6
}

# 메시 추가
opacity_map = np.full(threshed.n_cells, 0.1)  # 초기 투명도 설정
threshed.cell_data['opacity'] = opacity_map
mesh = p.add_mesh(threshed, scalars='opacity', cmap=custom_cmap, show_scalar_bar=True, scalar_bar_args=sargs, show_edges=True, opacity=1)

# 텍스트 추가
text_actor = p.add_text("Selected Program: None", position='upper_right', font_size=12, color='black')

# 슬라이더 콜백 함수
def slider_callback(value):
    selected_program = int(value)
    mask = threshed.cell_data['Agents'] == selected_program
    opacity = np.full(threshed.n_cells, 0.1)  # 기본 투명도 설정
    opacity[mask] = 1.0  # 선택된 프로그램에 높은 투명도 부여

    # 투명도 업데이트
    threshed.cell_data['opacity'] = opacity
    
    # 텍스트 업데이트
    text_actor.SetText(0, f"Selected Program: {annotations[selected_program]}")
    p.render()

# 슬라이더 위젯 추가
p.add_slider_widget(slider_callback, [0, len(program_prefs) - 1], value=0, title='Program Index')
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)

p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)
# 시각화 표시
p.show(jupyter_backend="trame", return_viewer=True)

Widget(value='<iframe src="http://localhost:62063/index.html?ui=P_0x1be1ab1d4d0_3&reconnect=auto" class="pyvis…

In [20]:
lattice_cens = init_avail_lattice.centroids_threshold(-1)

In [21]:
def distance(a_id, fns):
    fns_cens = lattice_cens[np.ravel_multi_index(np.array(fns).T, avail_lattice.shape)] # 이웃하는 격자의 중심점을 가져옴 (벡터화된 연산으로 개선)
    agn_avg_loc = np.average(agn_locs[a_id], axis=0)                                    # 에이전트의 평균 위치 계산 (루프 밖에서 한 번만 계산)
    dist_m = np.sqrt(((fns_cens - agn_avg_loc)**2).sum(axis=1))                         # 평균 위치와의 차이를 기반으로 거리 계산 (벡터화된 연산으로 개선)

    return dist_m  # 계산된 거리 반환

In [22]:
#square_weight = 0.3

cur_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)
frames = [cur_occ_lattice]

t = 0
n_frames = 50

# Precompute max_space for each agent to avoid recalculating in loop
max_spaces = np.rint(agent_areas).astype(int)

In [23]:
# 주어진 스텐실을 기반으로 이웃을 캐싱하는 함수
def cache_neighbors(lattice, stencil, stencil_sq):
    neighbors_cache = {}
    neighbors_sq_cache = {}
    for x in range(lattice.shape[0]):  # x축을 따라 루프
        for y in range(lattice.shape[1]):  # y축을 따라 루프
            for z in range(lattice.shape[2]):  # z축을 따라 루프
                loc = (x, y, z)  # 현재 위치를 (x, y, z)로 지정
                # 주어진 스텐실을 사용하여 현재 위치에서 이웃을 찾고 캐시에 저장
                neighbors_cache[loc] = lattice.find_neighbours_masked(stencil, loc=loc, id_type="3D")
                # 주어진 스텐실_sq를 사용하여 현재 위치에서 이웃을 찾고 캐시에 저장
                neighbors_sq_cache[loc] = lattice.find_neighbours_masked(stencil_sq, loc=loc, id_type="3D")
    return neighbors_cache, neighbors_sq_cache  # 두 개의 캐시를 반환

# 이웃 캐시 생성
neighbors_cache, neighbors_sq_cache = cache_neighbors(avail_lattice, stencil, stencil_sq)

In [24]:
def evaluate_ratio(a_locs, n, target_ratio):
    # 현재 위치와 새로운 위치의 비율을 계산
    new_locs = np.array(a_locs + [n])
    x_span = new_locs[:, 0].max() - new_locs[:, 0].min() + 1
    y_span = new_locs[:, 1].max() - new_locs[:, 1].min() + 1
    z_span = new_locs[:, 2].max() - new_locs[:, 2].min() + 1

    # 현재 스팬 비율을 계산하고 목표 비율과 비교
    current_ratio = (x_span / y_span, y_span / z_span, x_span / z_span)
    ratio_score = np.sum([abs(r - tr) for r, tr in zip(current_ratio, target_ratio)])
    
    return ratio_score

target_ratio = (1, 3,0.1)

In [25]:
last_move_direction = {a_id: None for a_id in program_prefs.index}


 ###-------------------------------------------------------------------------------------------------###

while t < n_frames:  # 프레임 수 만큼 반복
    n_fns = 0  # 빈 이웃이 없는 에이전트의 수를 세는 변수 초기화

    for a_id, a_prefs in program_prefs.iterrows():  # 각 에이전트의 선호도에 대해 반복
        # agn_locs[a_id]가 리스트가 아닌 경우 리스트로 변환
        if isinstance(agn_locs[a_id], (tuple, int)):
            agn_locs[a_id] = [agn_locs[a_id]]

        # 현재 에이전트의 위치를 가져옴
        a_locs = agn_locs[a_id]
        a_locs = [np.unravel_index(loc, avail_lattice.shape) if isinstance(loc, int) else loc for loc in a_locs]

        free_neighs = []  # 빈 이웃 위치를 저장할 리스트 초기화
        free_neighs_sq = []  # 빈 이웃 위치를 저장할 리스트 초기화

        # 에이전트의 각 위치에 대한 루프
        for a_loc in a_locs:
            if avail_lattice.shape[0] > 0 and avail_lattice.shape[1] > 0 and avail_lattice.shape[2] > 0:  # 격자 크기가 유효한 경우
                neighs = neighbors_cache[tuple(a_loc)]        # 현재 위치의 이웃을 가져옴
                neighs_sq = neighbors_sq_cache[tuple(a_loc)]  # 현재 위치의 이웃을 가져옴
            else:
                print("avail_lattice shape is invalid:", avail_lattice.shape)
                neighs = []     # 유효하지 않으면 빈 리스트로 설정
                neighs_sq = []  # 유효하지 않으면 빈 리스트로 설정

            # 빈 이웃 목록에 추가
            for n in neighs:
                if avail_lattice[tuple(n)]:       # 이웃이 비어 있으면
                    free_neighs.append(tuple(n))  # 튜플로 변환하여 추가

            for n in neighs_sq:
                if avail_lattice[tuple(n)]:          # 이웃이 비어 있으면
                    free_neighs_sq.append(tuple(n))  # 튜플로 변환하여 추가

        if not free_neighs:  # 빈 이웃이 없는 경우
            n_fns += 1

        ###-------------------------------------------------------------------------------------------------###

        if free_neighs:  # 빈 이웃이 있는 경우
            fns = np.array(free_neighs)  # 빈 이웃을 배열로 변환

            # 평가 함수 계산
            a_eval = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)

            for s in program_mtx.columns:
                s = int(s)
                vals = distance(s, fns)  # 거리 값을 계산
                a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
                a_eval += a_weighted_vals  # 평가 값에 추가

            free_neighs_set = set(free_neighs)  # 빈 이웃을 집합으로 변환
            free_neighs_sq_set = set(free_neighs_sq)  # 빈 이웃을 집합으로 변환
            free_neighs_count = [sum(1 for n in free_neighs_sq if n in free_neighs_set) for _ in free_neighs]

            # 수평 이동을 선호하도록 가중치 적용
            horizontal_bias = 0.9  # 수평 방향으로의 가중치 설정
            vertical_penalty = 0.1  # 수직 방향 이동에 대한 패널티 설정
            for i, n in enumerate(free_neighs):
                if abs(n[1] - a_locs[0][1]) > 0:  # y축 방향으로 이동
                    a_eval[i] *= horizontal_bias
                if abs(n[2] - a_locs[0][2]) > 0:  # z축 방향으로 이동
                    a_eval[i] *= vertical_penalty

            # square_weight = 1.0  # 수평 이동을 더 선호하도록 가중치 설정
            # for i, n in enumerate(free_neighs):
            #     if n in free_neighs_sq_set:
            #         a_eval[i] *= square_weight

            # 비율 평가 함수 적용
            for i, n in enumerate(free_neighs):
                ratio_score = evaluate_ratio(a_locs, n, target_ratio)
                a_eval[i] /= (1 + ratio_score)  # 비율 점수가 낮을수록 선호되도록 설정



            ###-------------------------------------------------------------------------------------------------###

            current_length = len(a_locs)
            i_eval = np.zeros(current_length)
            max_space = np.rint(agent_areas[a_id]).astype(int)  # 최대 공간 크기 계산

            if current_length >= max_space:  # 현재 길이가 최대 공간 이상인 경우
                i_eval = np.ones(current_length)

                for f in program_prefs.columns:
                    vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]
                    a_weighted_vals = vals * a_prefs[f]
                    i_eval *= a_weighted_vals

                for s in program_mtx.columns:
                    s = int(s)
                    vals = distance(s, np.array(a_locs))
                    a_weighted_vals = vals * program_mtx.loc[a_id, s]
                    i_eval *= a_weighted_vals

                i_neighs_count = np.zeros(current_length)

                for id, a_loc in enumerate(a_locs):
                    neighs = neighbors_cache[tuple(a_loc)]
                    for n in neighs:
                        if avail_lattice[tuple(n)] and (occ_lattice[tuple(n)] == a_id):
                            i_neighs_count[id] += 1
                

            selected_int_inner = np.argmin(i_eval)  # 내부 선택 인덱스 계산
            selected_int = np.argmax(a_eval)  # 외부 선택 인덱스 계산

            if current_length >= max_space and i_eval[selected_int_inner] < a_eval[selected_int]:  # 공간이 꽉 차고 내부 평가가 외부 평가보다 낮은 경우
                selected_inner_3d_id = tuple(a_locs[selected_int_inner])
                a_locs.pop(selected_int_inner)  # 내부 위치 제거
                avail_lattice[selected_inner_3d_id] = True
                occ_lattice[selected_inner_3d_id] = -1

            if current_length < max_space:  # 공간이 꽉 차지 않은 경우
                selected_neigh_3d_id = tuple(free_neighs[selected_int])
                a_locs.append(selected_neigh_3d_id)  # 빈 이웃에 추가
                avail_lattice[selected_neigh_3d_id] = False
                occ_lattice[selected_neigh_3d_id] = a_id


        agn_locs[a_id] = a_locs  # 업데이트된 위치 저장

    new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)  # 새로운 격자 생성
    frames.append(new_occ_lattice)  # 프레임에 추가
    print(t, "/", n_fns, end="  ")  # 현재 프레임과 빈 이웃이 없는 에이전트 수 출력
    t += 1  # 시간 증가

0 / 0  1 / 0  2 / 0  3 / 0  4 / 0  5 / 0  6 / 0  7 / 0  8 / 0  9 / 0  10 / 0  11 / 0  12 / 0  13 / 0  14 / 0  15 / 0  16 / 0  17 / 0  18 / 0  19 / 0  20 / 0  21 / 0  22 / 0  23 / 0  24 / 0  25 / 0  26 / 0  27 / 0  28 / 0  29 / 0  30 / 0  31 / 0  32 / 0  33 / 0  34 / 0  35 / 0  36 / 0  37 / 0  38 / 0  39 / 0  40 / 0  41 / 0  42 / 0  43 / 0  44 / 0  45 / 0  46 / 0  47 / 0  48 / 0  49 / 0  

In [26]:
# while t < n_frames:  # 프레임 수 만큼 반복
#     n_fns = 0  # 빈 이웃이 없는 에이전트의 수를 세는 변수 초기화

#     for a_id, a_prefs in program_prefs.iterrows():  # 각 에이전트의 선호도에 대해 반복
#         # agn_locs[a_id]가 리스트가 아닌 경우 리스트로 변환
#         if isinstance(agn_locs[a_id], (tuple, int)):
#             agn_locs[a_id] = [agn_locs[a_id]]

#         # 현재 에이전트의 위치를 가져옴
#         a_locs = agn_locs[a_id]
#         a_locs = [np.unravel_index(loc, avail_lattice.shape) if isinstance(loc, int) else loc for loc in a_locs]

#         free_neighs = []  # 빈 이웃 위치를 저장할 리스트 초기화
#         free_neighs_sq = []  # 빈 이웃 위치를 저장할 리스트 초기화

#         # 에이전트의 각 위치에 대한 루프
#         for a_loc in a_locs:
#             if avail_lattice.shape[0] > 0 and avail_lattice.shape[1] > 0 and avail_lattice.shape[2] > 0:  # 격자 크기가 유효한 경우
#                 neighs = neighbors_cache[tuple(a_loc)]        # 현재 위치의 이웃을 가져옴
#                 neighs_sq = neighbors_sq_cache[tuple(a_loc)]  # 현재 위치의 이웃을 가져옴
#             else:
#                 print("avail_lattice shape is invalid:", avail_lattice.shape)
#                 neighs = []     # 유효하지 않으면 빈 리스트로 설정
#                 neighs_sq = []  # 유효하지 않으면 빈 리스트로 설정

#             # 빈 이웃 목록에 추가
#             for n in neighs:
#                 if avail_lattice[tuple(n)]:       # 이웃이 비어 있으면
#                     free_neighs.append(tuple(n))  # 튜플로 변환하여 추가

#             for n in neighs_sq:
#                 if avail_lattice[tuple(n)]:          # 이웃이 비어 있으면
#                     free_neighs_sq.append(tuple(n))  # 튜플로 변환하여 추가

#         if not free_neighs:  # 빈 이웃이 없는 경우
#             n_fns += 1

#         if free_neighs:                  # 빈 이웃이 있는 경우
#             fns = np.array(free_neighs)  # 빈 이웃을 배열로 변환

#             # 평가 함수 계산
#             a_eval = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)

#             for s in program_mtx.columns:
#                 s = int(s)
#                 vals = distance(s, fns)  # 거리 값을 계산
#                 a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
#                 a_eval += a_weighted_vals  # 평가 값에 추가

#             free_neighs_set = set(free_neighs)  # 빈 이웃을 집합으로 변환
#             free_neighs_sq_set = set(free_neighs_sq)  # 빈 이웃을 집합으로 변환
#             free_neighs_count = [sum(1 for n in free_neighs_sq if n in free_neighs_set) for _ in free_neighs]

#             square_weight = 3.0  # 수평 이동을 선호하도록 가중치를 더욱 높게 설정
#             a_weighted_square = np.array(free_neighs_count) * square_weight  # 가중치 적용
#             a_eval *= a_weighted_square  # 평가 값에 추가

#             # 수평 이동을 선호하도록 가중치 적용
#             horizontal_bias = 5.0  # 수평 방향으로의 가중치 설정
#             vertical_penalty = 0.2  # 수직 방향 이동에 대한 강한 패널티 설정
#             for i, n in enumerate(free_neighs):
#                 if abs(n[1] - a_locs[0][1]) > 0:  # y축 방향으로 이동
#                     a_eval[i] *= horizontal_bias
#                 if abs(n[2] - a_locs[0][2]) > 0:  # z축 방향으로 이동
#                     a_eval[i] *= vertical_penalty

#             # 비율 평가 함수 적용
#             for i, n in enumerate(free_neighs):
#                 ratio_score = evaluate_ratio(a_locs, n, target_ratio)
#                 a_eval[i] /= (1 + ratio_score)  # 비율 점수가 낮을수록 선호되도록 설정

#             current_length = len(a_locs)
#             i_eval = np.zeros(current_length)
#             max_space = np.rint(agent_areas[a_id]).astype(int)  # 최대 공간 크기 계산

#             if current_length >= max_space:  # 현재 길이가 최대 공간 이상인 경우
#                 i_eval = np.ones(current_length)

#                 for f in program_prefs.columns:
#                     vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]
#                     a_weighted_vals = vals * a_prefs[f]
#                     i_eval *= a_weighted_vals

#                 for s in program_mtx.columns:
#                     s = int(s)
#                     vals = distance(s, np.array(a_locs))
#                     a_weighted_vals = vals * program_mtx.loc[a_id, s]
#                     i_eval *= a_weighted_vals

#                 i_neighs_count = np.zeros(current_length)

#                 for id, a_loc in enumerate(a_locs):
#                     neighs = neighbors_cache[tuple(a_loc)]
#                     for n in neighs:
#                         if avail_lattice[tuple(n)] and (occ_lattice[tuple(n)] == a_id):
#                             i_neighs_count[id] += 1

#                 i_weighted_square = i_neighs_count * square_weight
#                 i_eval *= i_weighted_square

#             selected_int_inner = np.argmin(i_eval)  # 내부 선택 인덱스 계산
#             selected_int = np.argmax(a_eval)  # 외부 선택 인덱스 계산

#             if current_length >= max_space and i_eval[selected_int_inner] < a_eval[selected_int]:  # 공간이 꽉 차고 내부 평가가 외부 평가보다 낮은 경우
#                 selected_inner_3d_id = tuple(a_locs[selected_int_inner])
#                 a_locs.pop(selected_int_inner)  # 내부 위치 제거
#                 avail_lattice[selected_inner_3d_id] = True
#                 occ_lattice[selected_inner_3d_id] = -1

#             if current_length < max_space:  # 공간이 꽉 차지 않은 경우
#                 selected_neigh_3d_id = tuple(free_neighs[selected_int])
#                 a_locs.append(selected_neigh_3d_id)  # 빈 이웃에 추가
#                 avail_lattice[selected_neigh_3d_id] = False
#                 occ_lattice[selected_neigh_3d_id] = a_id

#         agn_locs[a_id] = a_locs  # 업데이트된 위치 저장

#     new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)  # 새로운 격자 생성
#     frames.append(new_occ_lattice)  # 프레임에 추가
#     print(t, "/", n_fns, end="  ")  # 현재 프레임과 빈 이웃이 없는 에이전트 수 출력
#     t += 1  # 시간 증가

In [27]:
# # 주요 루프 최적화
# while t < n_frames:  # 프레임 수 만큼 반복
#     n_fns = 0        # 빈 이웃이 없는 에이전트의 수를 세는 변수 초기화

#     for a_id, a_prefs in program_prefs.iterrows():  # 각 에이전트의 선호도에 대해 반복
#         # agn_locs[a_id]가 리스트가 아닌 경우 리스트로 변환
#         if isinstance(agn_locs[a_id], (tuple, int)):
#             agn_locs[a_id] = [agn_locs[a_id]]

#         # 현재 에이전트의 위치를 가져옴
#         a_locs = agn_locs[a_id]
#         a_locs = [np.unravel_index(loc, avail_lattice.shape) if isinstance(loc, int) else loc for loc in a_locs]

#         free_neighs = []  # 빈 이웃 위치를 저장할 리스트 초기화
#         free_neighs_sq = []  # 빈 이웃 위치를 저장할 리스트 초기화

#         # 에이전트의 각 위치에 대한 루프
#         for a_loc in a_locs:
#             if avail_lattice.shape[0] > 0 and avail_lattice.shape[1] > 0 and avail_lattice.shape[2] > 0:  # 격자 크기가 유효한 경우
#                 neighs = neighbors_cache[tuple(a_loc)]        # 현재 위치의 이웃을 가져옴
#                 neighs_sq = neighbors_sq_cache[tuple(a_loc)]  # 현재 위치의 이웃을 가져옴
#             else:
#                 print("avail_lattice shape is invalid:", avail_lattice.shape)
#                 neighs = []     # 유효하지 않으면 빈 리스트로 설정
#                 neighs_sq = []  # 유효하지 않으면 빈 리스트로 설정

#             # 빈 이웃 목록에 추가
#             for n in neighs:
#                 if avail_lattice[tuple(n)]:       # 이웃이 비어 있으면
#                     free_neighs.append(tuple(n))  # 튜플로 변환하여 추가

#             for n in neighs_sq:
#                 if avail_lattice[tuple(n)]:          # 이웃이 비어 있으면
#                     free_neighs_sq.append(tuple(n))  # 튜플로 변환하여 추가

#         if not free_neighs:              # 빈 이웃이 없는 경우
#             n_fns += 1

#         if free_neighs:                  # 빈 이웃이 있는 경우
#             fns = np.array(free_neighs)  # 빈 이웃을 배열로 변환

#             # 평가 함수 계산
#             a_eval = np.prod([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)

#             for s in program_mtx.columns:
#                 s = int(s)
#                 vals = distance(s, fns)  # 거리 값을 계산
#                 a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
#                 a_eval += a_weighted_vals  # 평가 값에 추가

#             free_neighs_set = set(free_neighs)  # 빈 이웃을 집합으로 변환
#             free_neighs_sq_set = set(free_neighs_sq)  # 빈 이웃을 집합으로 변환
#             free_neighs_count = [sum(1 for n in free_neighs_sq if n in free_neighs_set) for _ in free_neighs]

#             square_weight = 1.2  # 가중치를 높게 설정하여 수평 이동 선호를 유도
#             a_weighted_square = np.array(free_neighs_count) * square_weight  # 가중치 적용
#             a_eval += a_weighted_square  # 평가 값에 추가

#             # # 빈 이웃 리스트를 무작위로 섞어서 특정 방향을 선호하지 않도록 조정
#             # np.random.shuffle(free_neighs)
#             # fns = np.array(free_neighs)  # 다시 배열로 변환

#              # 수평 이동을 선호하도록 가중치 적용
#             horizontal_bias = 2.0  # 수평 방향으로의 가중치 설정
#             vertical_penalty = 0.5  # 수직 방향 이동에 대한 패널티 설정
#             for i, n in enumerate(free_neighs):
#                 if abs(n[1] - a_locs[0][1]) > 0:  # y축 방향으로 이동
#                     a_eval[i] *= horizontal_bias
#                 if abs(n[2] - a_locs[0][2]) > 0:  # z축 방향으로 이동
#                     a_eval[i] *= vertical_penalty


#             # Check for squareness: Prefer moves that keep the space more square-like
#             for i, n in enumerate(free_neighs):
#                 dx = abs(n[0] - a_locs[0][0])
#                 dy = abs(n[1] - a_locs[0][1])
#                 dz = abs(n[2] - a_locs[0][2])
#                 if dx > 0 and dy > 0:  # If moving horizontally in both x and y
#                     a_eval[i] *= 1.5
#                 if dx > 0 and dz > 0:  # If moving horizontally in x and vertically in z
#                     a_eval[i] *= 0.5
                    
#             current_length = len(a_locs)
#             i_eval = np.zeros(current_length)
#             max_space = np.rint(agent_areas[a_id]).astype(int)  # 최대 공간 크기 계산

#             if current_length >= max_space:  # 현재 길이가 최대 공간 이상인 경우
#                 i_eval = np.ones(current_length)

#                 for f in program_prefs.columns:
#                     vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]
#                     a_weighted_vals = vals * a_prefs[f]
#                     i_eval *= a_weighted_vals

#                 for s in program_mtx.columns:
#                     s = int(s)
#                     vals = distance(s, np.array(a_locs))
#                     a_weighted_vals = vals * program_mtx.loc[a_id, s]
#                     i_eval *= a_weighted_vals

#                 i_neighs_count = np.zeros(current_length)

#                 for id, a_loc in enumerate(a_locs):
#                     neighs = neighbors_cache[tuple(a_loc)]
#                     for n in neighs:
#                         if avail_lattice[tuple(n)] and (occ_lattice[tuple(n)] == a_id):
#                             i_neighs_count[id] += 1

#                 i_weighted_square = i_neighs_count * square_weight
#                 i_eval *= i_weighted_square

#             selected_int_inner = np.argmin(i_eval)  # 내부 선택 인덱스 계산
#             selected_int = np.argmax(a_eval)  # 외부 선택 인덱스 계산

#             if current_length >= max_space and i_eval[selected_int_inner] < a_eval[selected_int]:  # 공간이 꽉 차고 내부 평가가 외부 평가보다 낮은 경우
#                 selected_inner_3d_id = tuple(a_locs[selected_int_inner])
#                 a_locs.pop(selected_int_inner)  # 내부 위치 제거
#                 avail_lattice[selected_inner_3d_id] = True
#                 occ_lattice[selected_inner_3d_id] = -1

#             if current_length < max_space:  # 공간이 꽉 차지 않은 경우
#                 selected_neigh_3d_id = tuple(free_neighs[selected_int])
#                 a_locs.append(selected_neigh_3d_id)  # 빈 이웃에 추가
#                 avail_lattice[selected_neigh_3d_id] = False
#                 occ_lattice[selected_neigh_3d_id] = a_id

#         agn_locs[a_id] = a_locs  # 업데이트된 위치 저장

#     new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)  # 새로운 격자 생성
#     frames.append(new_occ_lattice)  # 프레임에 추가
#     print(t, "/", n_fns, end="  ")  # 현재 프레임과 빈 이웃이 없는 에이전트 수 출력
#     t += 1  # 시간 증가


In [28]:

# # while 루프 시작
# while t < n_frames:
#     n_fns = 0  # 빈 이웃이 없는 에이전트의 수를 세는 변수 초기화

#     for a_id, a_prefs in program_prefs.iterrows():  # 각 에이전트에 대한 루프 시작
#         # agn_locs[a_id]가 리스트가 아닌 경우 리스트로 변환
#         if isinstance(agn_locs[a_id], tuple) or isinstance(agn_locs[a_id], int):
#             agn_locs[a_id] = [agn_locs[a_id]]

#         a_locs = agn_locs[a_id]  # 현재 에이전트의 위치를 가져옴
#         #print(f"Agent {a_id} locations: {a_locs}")  # 각 에이전트의 위치 출력

#         # 모든 위치가 3D 좌표인지 확인하고, 필요한 경우 변환
#         a_locs = [np.unravel_index(loc, avail_lattice.shape) if isinstance(loc, int) else loc for loc in a_locs]
#         print(f"Converted Agent {a_id} locations: {a_locs}")  # 변환된 위치 출력

#         free_neighs = []  # 빈 이웃 목록 초기화
#         free_neighs_sq = []  # 빈 이웃(제곱 거리 내의) 목록 초기화

#         for a_loc in a_locs:  # 에이전트의 각 위치에 대한 루프
#             #print(f"Processing location: {a_loc}")  # 디버깅을 위한 위치 출력
#             #print(f"avail_lattice shape: {avail_lattice.shape}")  # 디버깅을 위한 avail_lattice 형상 출력

#             if avail_lattice.shape[0] > 0 and avail_lattice.shape[1] > 0 and avail_lattice.shape[2] > 0:
#                 neighs = avail_lattice.find_neighbours_masked(stencil, loc=a_loc)  # 현재 위치에서 stencil을 사용하여 가능한 이웃 찾기
#                 neighs_sq = avail_lattice.find_neighbours_masked(stencil_sq, loc=a_loc)  # 현재 위치에서 stencil_sq를 사용하여 가능한 이웃 찾기
#             else:
#                 print("avail_lattice shape is invalid:", avail_lattice.shape)
#                 neighs = []
#                 neighs_sq = []

#             for n in neighs:  # 각 이웃에 대해
#                 neigh_3d_id = np.unravel_index(n, avail_lattice.shape)  # 이웃의 3D 좌표를 얻음
#                 if avail_lattice[neigh_3d_id]:  # 이웃이 사용 가능하면
#                     free_neighs.append(neigh_3d_id)  # 빈 이웃 목록에 추가

#             for n in neighs_sq:  # 제곱 stencil의 각 이웃에 대해
#                 neigh_3d_id = np.unravel_index(n, avail_lattice.shape)  # 이웃의 3D 좌표를 얻음
#                 if avail_lattice[neigh_3d_id]:  # 이웃이 사용 가능하면
#                     free_neighs_sq.append(neigh_3d_id)  # 빈 이웃(제곱 거리 내의) 목록에 추가

#         if not len(free_neighs) > 0:  # 빈 이웃이 없는 경우
#             n_fns += 1  # 빈 이웃이 없는 에이전트 수 증가

#         if len(free_neighs) > 0:  # 빈 이웃이 있는 경우
#             fns = np.array(free_neighs)  # 빈 이웃 목록을 numpy 배열로 변환

#             # 평가 함수 계산
#             a_eval = np.prod([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] ** a_prefs[f] for f in program_prefs.columns], axis=0)

#             for s in program_mtx.columns:  # 각 프로그램 매트릭스 열에 대해
#                 s = int(s)
#                 vals = distance(s, fns)  # 거리 계산
#                 a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
#                 a_eval *= a_weighted_vals  # 평가 값에 곱셈

#             free_neighs_count = []  # 빈 이웃의 수를 세는 목록 초기화

#             for free_neigh in free_neighs:  # 각 빈 이웃에 대해
#                 free_neighs_count.append(free_neighs_sq.count(free_neigh))  # 빈 이웃(제곱 거리 내의) 수를 셈

#             # 가중치를 0.5로 조정
#             square_weight = 0.5
#             a_weighted_square = np.array(free_neighs_count) * square_weight  # 가중치를 적용한 제곱
#             a_eval *= a_weighted_square  # 평가 값에 곱셈

#             current_length = np.copy(len(a_locs))  # 현재 위치의 수 복사

#             i_eval = np.zeros(current_length)  # 초기 내부 평가 값 설정
#             max_space_raw = agent_areas[a_id]  # 에이전트의 최대 공간 설정
#             max_space = np.rint(max_space_raw)  # 최대 공간을 반올림

#             if current_length >= max_space:  # 현재 위치의 수가 최대 공간 이상인 경우
#                 i_eval = np.ones(current_length)  # 내부 평가 값을 모두 1로 설정

#                 for f in program_prefs.columns:  # 각 프로그램 선호도 열에 대해
#                     vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]  # 필드 값 가져오기
#                     a_weighted_vals = vals ** a_prefs[f]  # 가중치 적용
#                     i_eval *= a_weighted_vals  # 내부 평가 값에 곱셈

#                 for s in program_mtx.columns:  # 각 프로그램 매트릭스 열에 대해
#                     s = int(s)
#                     vals = distance(s, np.array(a_locs))  # 거리 계산
#                     a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
#                     i_eval *= a_weighted_vals  # 내부 평가 값에 곱셈

#                 i_neighs_count = np.zeros(current_length)  # 내부 이웃 수 초기화

#                 for id, a_loc in enumerate(a_locs):  # 각 위치에 대해
#                     neighs = init_avail_lattice.find_neighbours_masked(stencil, loc=a_loc)  # 초기 사용 가능 격자에서 이웃 찾기
#                     for n in neighs:  # 각 이웃에 대해
#                         neigh_3d_id = np.unravel_index(n, avail_lattice.shape)  # 이웃의 3D 좌표 얻기
#                         i_neighs_count[id] += (occ_lattice == a_id)[neigh_3d_id]  # 이웃의 점유 상태 확인 및 수 증가

#                 i_weighted_square = np.array(i_neighs_count) * square_weight  # 가중치를 적용한 제곱
#                 i_eval *= i_weighted_square  # 내부 평가 값에 곱셈

#             selected_int_inner = np.argmin(i_eval)  # 내부 평가 값이 최소인 인덱스 선택
#             selected_int = np.argmax(a_eval)  # 외부 평가 값이 최대인 인덱스 선택

#             if (current_length >= max_space) and i_eval[selected_int_inner] < a_eval[selected_int]:  # 최대 공간을 넘고 내부 평가 값이 외부 평가 값보다 작은 경우
#                 selected_inner_3d_id = tuple(a_locs[selected_int_inner])  # 선택된 내부 위치의 3D 좌표
#                 selected_inner_loc = a_locs[selected_int_inner]  # 선택된 내부 위치
#                 a_locs.pop(selected_int_inner)  # 에이전트 위치 목록에서 제거
#                 avail_lattice[selected_inner_3d_id] = 1  # 해당 위치를 사용 가능으로 표시
#                 occ_lattice[selected_inner_3d_id] = -1  # 해당 위치를 비어 있음으로 표시

#             if current_length < max_space:  # 현재 위치의 수가 최대 공간보다 작은 경우
#                 selected_neigh_3d_id = free_neighs[selected_int]  # 선택된 이웃의 3D 좌표
#                 selected_neigh_loc = np.array(selected_neigh_3d_id).flatten()  # 선택된 이웃 위치
#                 a_locs.append(selected_neigh_loc)  # 에이전트 위치 목록에 추가
#                 avail_lattice[selected_neigh_3d_id] = 0  # 해당 위치를 사용 중으로 표시
#                 occ_lattice[selected_neigh_3d_id] = a_id  # 해당 위치를 에이전트 ID로 표시

#         # 에이전트의 수정된 위치를 다시 할당
#         agn_locs[a_id] = a_locs

#     new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)  # 새로운 점유 격자 생성
#     frames.append(new_occ_lattice)  # 프레임 목록에 추가
#     print(t, "/", n_fns, end="  ")  # 현재 시간 단계와 빈 이웃이 없는 에이전트 수 출력
#     t += 1  # 시간 단계 증가

In [29]:
# import numpy as np

# # Max spaces for each agent precomputed
# max_spaces = np.rint(agent_areas).astype(int)

# # Cache the neighbors for each position in the lattice to avoid recalculating
# neighbors_cache = {}
# neighbors_sq_cache = {}

# # Precompute neighbors for all locations in the lattice
# for x in range(avail_lattice.shape[0]):
#     for y in range(avail_lattice.shape[1]):
#         for z in range(avail_lattice.shape[2]):
#             loc = (x, y, z)
#             neighbors_cache[loc] = avail_lattice.find_neighbours_masked(stencil, loc=loc)
#             neighbors_sq_cache[loc] = avail_lattice.find_neighbours_masked(stencil_sq, loc=loc)

# # while 루프 시작
# while t < n_frames:
#     n_fns = 0  # 빈 이웃이 없는 에이전트의 수를 세는 변수 초기화

#     for a_id, a_prefs in program_prefs.iterrows():  # 각 에이전트에 대한 루프 시작
#         # agn_locs[a_id]가 리스트가 아닌 경우 리스트로 변환
#         if isinstance(agn_locs[a_id], tuple) or isinstance(agn_locs[a_id], int):
#             agn_locs[a_id] = [agn_locs[a_id]]

#         a_locs = agn_locs[a_id]  # 현재 에이전트의 위치를 가져옴
#         #print(f"Agent {a_id} locations: {a_locs}")  # 각 에이전트의 위치 출력

#         # 모든 위치가 3D 좌표인지 확인하고, 필요한 경우 변환
#         a_locs = [np.unravel_index(loc, avail_lattice.shape) if isinstance(loc, int) else loc for loc in a_locs]
#         print(f"Converted Agent {a_id} locations: {a_locs}")  # 변환된 위치 출력

#         free_neighs = []  # 빈 이웃 목록 초기화
#         free_neighs_sq = []  # 빈 이웃(제곱 거리 내의) 목록 초기화

#         for a_loc in a_locs:  # 에이전트의 각 위치에 대한 루프
#             #print(f"Processing location: {a_loc}")  # 디버깅을 위한 위치 출력
#             #print(f"avail_lattice shape: {avail_lattice.shape}")  # 디버깅을 위한 avail_lattice 형상 출력

#             if avail_lattice.shape[0] > 0 and avail_lattice.shape[1] > 0 and avail_lattice.shape[2] > 0:
#                 neighs = avail_lattice.find_neighbours_masked(stencil, loc=a_loc)  # 현재 위치에서 stencil을 사용하여 가능한 이웃 찾기
#                 neighs_sq = avail_lattice.find_neighbours_masked(stencil_sq, loc=a_loc)  # 현재 위치에서 stencil_sq를 사용하여 가능한 이웃 찾기
#             else:
#                 print("avail_lattice shape is invalid:", avail_lattice.shape)
#                 neighs = []
#                 neighs_sq = []

#             for n in neighs:  # 각 이웃에 대해
#                 neigh_3d_id = np.unravel_index(n, avail_lattice.shape)  # 이웃의 3D 좌표를 얻음
#                 if avail_lattice[neigh_3d_id]:  # 이웃이 사용 가능하면
#                     free_neighs.append(neigh_3d_id)  # 빈 이웃 목록에 추가

#             for n in neighs_sq:  # 제곱 stencil의 각 이웃에 대해
#                 neigh_3d_id = np.unravel_index(n, avail_lattice.shape)  # 이웃의 3D 좌표를 얻음
#                 if avail_lattice[neigh_3d_id]:  # 이웃이 사용 가능하면
#                     free_neighs_sq.append(neigh_3d_id)  # 빈 이웃(제곱 거리 내의) 목록에 추가

#         if not len(free_neighs) > 0:  # 빈 이웃이 없는 경우
#             n_fns += 1  # 빈 이웃이 없는 에이전트 수 증가

#         if len(free_neighs) > 0:  # 빈 이웃이 있는 경우
#             fns = np.array(free_neighs)  # 빈 이웃 목록을 numpy 배열로 변환

#             # 평가 함수 계산
#             a_eval = np.prod([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] ** a_prefs[f] for f in program_prefs.columns], axis=0)

#             for s in program_mtx.columns:  # 각 프로그램 매트릭스 열에 대해
#                 s = int(s)
#                 vals = distance(s, fns)  # 거리 계산
#                 a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
#                 a_eval *= a_weighted_vals  # 평가 값에 곱셈

#             free_neighs_count = []  # 빈 이웃의 수를 세는 목록 초기화

#             for free_neigh in free_neighs:  # 각 빈 이웃에 대해
#                 free_neighs_count.append(free_neighs_sq.count(free_neigh))  # 빈 이웃(제곱 거리 내의) 수를 셈

#             # 가중치를 0.5로 조정
#             square_weight = 0.5
#             a_weighted_square = np.array(free_neighs_count) * square_weight  # 가중치를 적용한 제곱
#             a_eval *= a_weighted_square  # 평가 값에 곱셈

#             current_length = np.copy(len(a_locs))  # 현재 위치의 수 복사

#             i_eval = np.zeros(current_length)  # 초기 내부 평가 값 설정
#             max_space_raw = agent_areas[a_id]  # 에이전트의 최대 공간 설정
#             max_space = np.rint(max_space_raw)  # 최대 공간을 반올림

#             if current_length >= max_space:  # 현재 위치의 수가 최대 공간 이상인 경우
#                 i_eval = np.ones(current_length)  # 내부 평가 값을 모두 1로 설정

#                 for f in program_prefs.columns:  # 각 프로그램 선호도 열에 대해
#                     vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]  # 필드 값 가져오기
#                     a_weighted_vals = vals ** a_prefs[f]  # 가중치 적용
#                     i_eval *= a_weighted_vals  # 내부 평가 값에 곱셈

#                 for s in program_mtx.columns:  # 각 프로그램 매트릭스 열에 대해
#                     s = int(s)
#                     vals = distance(s, np.array(a_locs))  # 거리 계산
#                     a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
#                     i_eval *= a_weighted_vals  # 내부 평가 값에 곱셈

#                 i_neighs_count = np.zeros(current_length)  # 내부 이웃 수 초기화

#                 for id, a_loc in enumerate(a_locs):  # 각 위치에 대해
#                     neighs = init_avail_lattice.find_neighbours_masked(stencil, loc=a_loc)  # 초기 사용 가능 격자에서 이웃 찾기
#                     for n in neighs:  # 각 이웃에 대해
#                         neigh_3d_id = np.unravel_index(n, avail_lattice.shape)  # 이웃의 3D 좌표 얻기
#                         i_neighs_count[id] += (occ_lattice == a_id)[neigh_3d_id]  # 이웃의 점유 상태 확인 및 수 증가

#                 i_weighted_square = np.array(i_neighs_count) * square_weight  # 가중치를 적용한 제곱
#                 i_eval *= i_weighted_square  # 내부 평가 값에 곱셈

#             selected_int_inner = np.argmin(i_eval)  # 내부 평가 값이 최소인 인덱스 선택
#             selected_int = np.argmax(a_eval)  # 외부 평가 값이 최대인 인덱스 선택

#             if (current_length >= max_space) and i_eval[selected_int_inner] < a_eval[selected_int]:  # 최대 공간을 넘고 내부 평가 값이 외부 평가 값보다 작은 경우
#                 selected_inner_3d_id = tuple(a_locs[selected_int_inner])  # 선택된 내부 위치의 3D 좌표
#                 selected_inner_loc = a_locs[selected_int_inner]  # 선택된 내부 위치
#                 a_locs.pop(selected_int_inner)  # 에이전트 위치 목록에서 제거
#                 avail_lattice[selected_inner_3d_id] = 1  # 해당 위치를 사용 가능으로 표시
#                 occ_lattice[selected_inner_3d_id] = -1  # 해당 위치를 비어 있음으로 표시

#             if current_length < max_space:  # 현재 위치의 수가 최대 공간보다 작은 경우
#                 selected_neigh_3d_id = free_neighs[selected_int]  # 선택된 이웃의 3D 좌표
#                 selected_neigh_loc = np.array(selected_neigh_3d_id).flatten()  # 선택된 이웃 위치
#                 a_locs.append(selected_neigh_loc)  # 에이전트 위치 목록에 추가
#                 avail_lattice[selected_neigh_3d_id] = 0  # 해당 위치를 사용 중으로 표시
#                 occ_lattice[selected_neigh_3d_id] = a_id  # 해당 위치를 에이전트 ID로 표시

#         # 에이전트의 수정된 위치를 다시 할당
#         agn_locs[a_id] = a_locs

#     new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)  # 새로운 점유 격자 생성
#     frames.append(new_occ_lattice)  # 프레임 목록에 추가
#     print(t, "/", n_fns, end="  ")  # 현재 시간 단계와 빈 이웃이 없는 에이전트 수 출력
#     t += 1  # 시간 단계 증가

In [30]:
# while t < n_frames:
#     n_fns = 0  # 빈 이웃이 없는 에이전트의 수를 세는 변수 초기화

#     for a_id, a_prefs in program_prefs.iterrows():  # 각 에이전트에 대한 루프 시작
#         # agn_locs[a_id]가 리스트가 아닌 경우 리스트로 변환
#         if isinstance(agn_locs[a_id], tuple) or isinstance(agn_locs[a_id], int):
#             agn_locs[a_id] = [agn_locs[a_id]]

#         a_locs = agn_locs[a_id]  # 현재 에이전트의 위치를 가져옴
#         print(f"Agent {a_id} locations: {a_locs}")  # 각 에이전트의 위치 출력

#         # 모든 위치가 3D 좌표인지 확인하고, 필요한 경우 변환
#         a_locs = [np.unravel_index(loc, avail_lattice.shape) if isinstance(loc, int) else loc for loc in a_locs]
#         print(f"Converted Agent {a_id} locations: {a_locs}")  # 변환된 위치 출력

#         free_neighs = []  # 빈 이웃 목록 초기화
#         free_neighs_sq = []  # 빈 이웃(제곱 거리 내의) 목록 초기화

#         for a_loc in a_locs:  # 에이전트의 각 위치에 대한 루프
#             #print(f"Processing location: {a_loc}")  # 디버깅을 위한 위치 출력
#             #print(f"avail_lattice shape: {avail_lattice.shape}")  # 디버깅을 위한 avail_lattice 형상 출력

#             if avail_lattice.shape[0] > 0 and avail_lattice.shape[1] > 0 and avail_lattice.shape[2] > 0:
#                 neighs = avail_lattice.find_neighbours_masked(stencil, loc=a_loc)  # 현재 위치에서 stencil을 사용하여 가능한 이웃 찾기
#                 neighs_sq = avail_lattice.find_neighbours_masked(stencil_sq, loc=a_loc)  # 현재 위치에서 stencil_sq를 사용하여 가능한 이웃 찾기
#             else:
#                 print("avail_lattice shape is invalid:", avail_lattice.shape)
#                 neighs = []
#                 neighs_sq = []

#             for n in neighs:  # 각 이웃에 대해
#                 neigh_3d_id = np.unravel_index(n, avail_lattice.shape)  # 이웃의 3D 좌표를 얻음
#                 if avail_lattice[neigh_3d_id]:  # 이웃이 사용 가능하면
#                     free_neighs.append(neigh_3d_id)  # 빈 이웃 목록에 추가

#             for n in neighs_sq:  # 제곱 stencil의 각 이웃에 대해
#                 neigh_3d_id = np.unravel_index(n, avail_lattice.shape)  # 이웃의 3D 좌표를 얻음
#                 if avail_lattice[neigh_3d_id]:  # 이웃이 사용 가능하면
#                     free_neighs_sq.append(neigh_3d_id)  # 빈 이웃(제곱 거리 내의) 목록에 추가

#         if not len(free_neighs) > 0:  # 빈 이웃이 없는 경우
#             n_fns += 1  # 빈 이웃이 없는 에이전트 수 증가

#         if len(free_neighs) > 0:  # 빈 이웃이 있는 경우
#             fns = np.array(free_neighs)  # 빈 이웃 목록을 numpy 배열로 변환

#             # 평가 함수 계산
#             a_eval = np.prod([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] ** a_prefs[f] for f in program_prefs.columns], axis=0)

#             for s in program_mtx.columns:  # 각 프로그램 매트릭스 열에 대해
#                 s = int(s)
#                 vals = distance(s, fns)  # 거리 계산
#                 a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
#                 a_eval *= a_weighted_vals  # 평가 값에 곱셈

#             free_neighs_count = []  # 빈 이웃의 수를 세는 목록 초기화

#             for free_neigh in free_neighs:  # 각 빈 이웃에 대해
#                 free_neighs_count.append(free_neighs_sq.count(free_neigh))  # 빈 이웃(제곱 거리 내의) 수를 셈

#             a_weighted_square = np.array(free_neighs_count) * square_weight  # 가중치를 적용한 제곱
#             a_eval *= a_weighted_square  # 평가 값에 곱셈

#             current_length = np.copy(len(a_locs))  # 현재 위치의 수 복사

#             i_eval = np.zeros(current_length)  # 초기 내부 평가 값 설정
#             max_space_raw = agent_areas[a_id]  # 에이전트의 최대 공간 설정
#             max_space = np.rint(max_space_raw)  # 최대 공간을 반올림

#             if current_length >= max_space:  # 현재 위치의 수가 최대 공간 이상인 경우
#                 i_eval = np.ones(current_length)  # 내부 평가 값을 모두 1로 설정

#                 for f in program_prefs.columns:  # 각 프로그램 선호도 열에 대해
#                     vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]  # 필드 값 가져오기
#                     a_weighted_vals = vals ** a_prefs[f]  # 가중치 적용
#                     i_eval *= a_weighted_vals  # 내부 평가 값에 곱셈

#                 for s in program_mtx.columns:  # 각 프로그램 매트릭스 열에 대해
#                     s = int(s)
#                     vals = distance(s, np.array(a_locs))  # 거리 계산
#                     a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
#                     i_eval *= a_weighted_vals  # 내부 평가 값에 곱셈

#                 i_neighs_count = np.zeros(current_length)  # 내부 이웃 수 초기화

#                 for id, a_loc in enumerate(a_locs):  # 각 위치에 대해
#                     neighs = init_avail_lattice.find_neighbours_masked(stencil, loc=a_loc)  # 초기 사용 가능 격자에서 이웃 찾기
#                     for n in neighs:  # 각 이웃에 대해
#                         neigh_3d_id = np.unravel_index(n, avail_lattice.shape)  # 이웃의 3D 좌표 얻기
#                         i_neighs_count[id] += (occ_lattice == a_id)[neigh_3d_id]  # 이웃의 점유 상태 확인 및 수 증가

#                 i_weighted_square = np.array(i_neighs_count) * square_weight  # 가중치를 적용한 제곱
#                 i_eval *= i_weighted_square  # 내부 평가 값에 곱셈

#             selected_int_inner = np.argmin(i_eval)  # 내부 평가 값이 최소인 인덱스 선택
#             selected_int = np.argmax(a_eval)  # 외부 평가 값이 최대인 인덱스 선택

#             if (current_length >= max_space) and i_eval[selected_int_inner] < a_eval[selected_int]:  # 최대 공간을 넘고 내부 평가 값이 외부 평가 값보다 작은 경우
#                 selected_inner_3d_id = tuple(a_locs[selected_int_inner])  # 선택된 내부 위치의 3D 좌표
#                 selected_inner_loc = a_locs[selected_int_inner]  # 선택된 내부 위치
#                 a_locs.pop(selected_int_inner)  # 에이전트 위치 목록에서 제거
#                 avail_lattice[selected_inner_3d_id] = 1  # 해당 위치를 사용 가능으로 표시
#                 occ_lattice[selected_inner_3d_id] = -1  # 해당 위치를 비어 있음으로 표시

#             if current_length < max_space:  # 현재 위치의 수가 최대 공간보다 작은 경우
#                 selected_neigh_3d_id = free_neighs[selected_int]  # 선택된 이웃의 3D 좌표
#                 selected_neigh_loc = np.array(selected_neigh_3d_id).flatten()  # 선택된 이웃 위치
#                 a_locs.append(selected_neigh_loc)  # 에이전트 위치 목록에 추가
#                 avail_lattice[selected_neigh_3d_id] = 0  # 해당 위치를 사용 중으로 표시
#                 occ_lattice[selected_neigh_3d_id] = a_id  # 해당 위치를 에이전트 ID로 표시

#         # 에이전트의 수정된 위치를 다시 할당
#         agn_locs[a_id] = a_locs

#     new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)  # 새로운 점유 격자 생성
#     frames.append(new_occ_lattice)  # 프레임 목록에 추가
#     print(t, "/", n_fns, end="  ")  # 현재 시간 단계와 빈 이웃이 없는 에이전트 수 출력
#     t += 1  # 시간 단계 증가

#     # 주석 처리된 CSV 저장 코드
#     # for i, lattice in enumerate(frames):
#     #     save_csv_path = os.path.relpath('/content/drive/MyDrive/topotry2/abm_mcda/abm_f_'+ f'{i:03}' + '.csv')
#     #     lattice.to_csv(save_csv_path)


In [31]:
from matplotlib.colors import ListedColormap

# PyVista 플로터 초기화
p = pv.Plotter(notebook=True)

# 그리드 설정
#base_lattice = frames[0]
grid = pv.ImageData()
grid.dimensions = np.array(occ_lattice.shape) + 1
# grid.origin과 grid.spacing 설정에 avail_lattice의 속성 사용
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# 데이터 할당
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)

# 필터링
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

# 초기 컬러맵 설정
initial_colors = ["#808080" if i == 0 else "#FF0000" for i in range(len(program_prefs))]
custom_cmap = ListedColormap(initial_colors)

# 스칼라 바 설정
annotations = {i: name for i, name in enumerate(space_list.values())}
sargs = {
    "shadow": True,
    "n_labels": len(program_prefs),
    "italic": False,
    "fmt": " %.0f",
    "font_family": "arial",
    "height": 1,
    "vertical": True,
    "title_font_size": 10,
    "label_font_size": 6
}

# 메시 추가
opacity_map = np.full(threshed.n_cells, 0.1)  # 초기 투명도 설정
threshed.cell_data['opacity'] = opacity_map
mesh = p.add_mesh(threshed, scalars='opacity', cmap=custom_cmap, show_scalar_bar=True, scalar_bar_args=sargs, show_edges=True, opacity=1)

# 텍스트 추가
text_actor = p.add_text("Selected Program: None", position='upper_right', font_size=12, color='black')

# 슬라이더 콜백 함수
def slider_callback(value):
    selected_program = int(value)
    mask = threshed.cell_data['Agents'] == selected_program
    opacity = np.full(threshed.n_cells, 0.1)  # 기본 투명도 설정
    opacity[mask] = 1.0  # 선택된 프로그램에 높은 투명도 부여

    # 투명도 업데이트
    threshed.cell_data['opacity'] = opacity
    
    # 텍스트 업데이트
    text_actor.SetText(0, f"Selected Program: {annotations[selected_program]}")
    p.render()

# 슬라이더 위젯 추가
p.add_slider_widget(slider_callback, [0, len(program_prefs) - 1], value=0, title='Program Index')
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)

p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)
# 시각화 표시
p.show(jupyter_backend="trame", return_viewer=True)

Widget(value='<iframe src="http://localhost:62063/index.html?ui=P_0x1be14939c10_4&reconnect=auto" class="pyvis…

In [ ]:
# import numpy as np

# # 가중치 설정
# program_prefs = {
#     'Lobby': {'sun_acc': 0.2, 'ent_acc': 1, 'str_acc': 1, 'ung_pre': 0.1, 'dist_façade': 0.1, 'top_pre': 0.1},
#     'External Contact': {'sun_acc': 0.2, 'ent_acc': 1, 'str_acc': 1, 'ung_pre': 0.2, 'dist_façade': 0.2, 'top_pre': 0.1}
# }

# # 가중치 적용을 보여주는 예제 함수
# def calculate_evaluation(free_neighs, fields, a_prefs):
#     a_eval = np.zeros(len(free_neighs))
#     for f in a_prefs.keys():
#         a_eval += fields[f] * a_prefs[f]
#     return a_eval

# # 빈 이웃 위치 (예시)
# free_neighs = np.array([[5, 5, 4], [5, 5, 6], [5, 4, 5], [5, 6, 5], [4, 5, 5], [6, 5, 5]])

# # 환경 정보 필드 (예시, 실제 데이터와 다를 수 있음)
# fields = {
#     'sun_acc': np.array([0.2, 0.2, 0.5, 0.3, 0.1, 0.1]),
#     'ent_acc': np.array([1, 1, 0.9, 0.8, 0.7, 0.6]),
#     'str_acc': np.array([1, 1, 0.8, 0.7, 0.6, 0.5]),
#     'ung_pre': np.array([0.1, 0.2, 0.3, 0.1, 0.2, 0.1]),
#     'dist_façade': np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6]),
#     'top_pre': np.array([0.1, 0.1, 0.2, 0.3, 0.4, 0.5])
# }

# # 로비 가중치 계산
# lobby_prefs = program_prefs['Lobby']
# lobby_eval = calculate_evaluation(free_neighs, fields, lobby_prefs)
# print("Lobby Evaluation: ", lobby_eval)

# # External Contact 가중치 계산
# external_contact_prefs = program_prefs['External Contact']
# external_contact_eval = calculate_evaluation(free_neighs, fields, external_contact_prefs)
# print("External Contact Evaluation: ", external_contact_eval)

# # 수평 및 수직 이동 가중치 적용 예시
# horizontal_bias = 3.0
# vertical_penalty = 0.2

# for i, n in enumerate(free_neighs):
#     if abs(n[1] - free_neighs[0][1]) > 0:  # y축 방향으로 이동
#         lobby_eval[i] += horizontal_bias
#         external_contact_eval[i] += horizontal_bias
#     if abs(n[2] - free_neighs[0][2]) > 0:  # z축 방향으로 이동
#         lobby_eval[i] -= vertical_penalty
#         external_contact_eval[i] -= vertical_penalty

# print("Lobby Evaluation after applying horizontal and vertical weights: ", lobby_eval)
# print("External Contact Evaluation after applying horizontal and vertical weights: ", external_contact_eval)


Exception raised
KeyError('4907836ca2fedc713ae3ebd7c7d8f013_681f')
Traceback (most recent call last):
  File "c:\Users\junglim\AppData\Local\Programs\Python\Python311\Lib\site-packages\wslink\protocol.py", line 308, in onCompleteMessage
    results = func(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\junglim\AppData\Local\Programs\Python\Python311\Lib\site-packages\trame_vtk\modules\vtk\protocols\local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "c:\Users\junglim\AppData\Local\Programs\Python\Python311\Lib\site-packages\trame_vtk\modules\vtk\serializers\synchronization_context.py", line 35, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
                ~~~~~~~~~~~~~~~~~~~~~^^^^^^^
KeyError: '4907836ca2fedc713ae3ebd7c7d8f013_681f'

Exception raised
KeyError('fb788328dc3082584ba7433c2807b4c5_1020L')
Traceback (most recent call last):
  File "c:\Users\junglim\AppData\Local\Programs\Pyt